In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/2_comments_youtube_refined.csv")

In [ ]:
# Muestreo balanceado
N = 9000

df_sampled = (
    df.groupby("condiciones_cuenta", group_keys=False)
    .apply(lambda x: x.sample(min(len(x), N), random_state=42))
    .reset_index(drop=True)
)

# Ordenar por bando y canal
df_sampled = df_sampled.sort_values(by=["condiciones_cuenta", "channel_title"]).reset_index(drop=True)

# Agregar columna para etiquetado manual
df_sampled["label_comentario"] = "" # ruso / neutro / ucraniano
df_sampled["es_sarcastico"] = "" # TRUE / FALSE
df_sampled["fuera_de_contexto"] = ""  # TRUE / FALSE


# Agregar ID único por fila
df_sampled["id_muestra"] = df_sampled.index + 1

# Guardar dataset limpio en la carpeta del proyecto
#csv_path = "../data/processed/comentarios_para_etiquetar_v2.csv"
excel_path = "../data/processed/comentarios_para_etiquetar_v2.xlsx"

# CSV con codificación compatible con Excel (acentos OK)
#df_sampled.to_csv(csv_path, index=False, encoding="utf-8-sig")

# También guardar como Excel
df_sampled.to_excel(excel_path, index=False)

print(f"Total de comentarios extraídos: {len(df_sampled)}")
#print(f"Archivos guardados en:\n- {csv_path}\n- {excel_path}")

C:\Users\User\AppData\Local\Temp\ipykernel_4824\706829373.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), N), random_state=42))


Total de comentarios extraídos: 9000


In [ ]:
df_ya_clasificado = pd.read_excel(excel_path)


In [29]:
df_ya_clasificado["label_comentario"].value_counts()


label_comentario
ruso         907
ucraniano    312
neutro       234
Name: count, dtype: int64

In [30]:
# Filtrar solo los ya etiquetados
df_etiquetados = df_ya_clasificado[df_ya_clasificado["label_comentario"].notna() & (df_ya_clasificado["label_comentario"] != "")].copy()

# Limpiar comment_id de ambos datasets
df_etiquetados.loc[:, "comment_id"] = df_etiquetados["comment_id"].astype(str).str.strip()
df.loc[:, "comment_id"] = df["comment_id"].astype(str).str.strip()

# Obtener los restantes
comentarios_existentes = set(df_etiquetados["comment_id"])
df_restante = df[~df["comment_id"].isin(comentarios_existentes)]

print(f"Comentarios ya etiquetados: {len(df_etiquetados)}")
print(f"Comentarios restantes para muestreo: {len(df_restante)}")


Comentarios ya etiquetados: 1453
Comentarios restantes para muestreo: 371109


# Detección del idioma

In [31]:
import langid

df["idioma"] = df["comment"].apply(lambda x: langid.classify(str(x))[0])


In [32]:
df["idioma"].value_counts().head(20)

idioma
es    297772
en     12367
pt      7982
zh      6339
an      4922
gl      3187
fi      2695
it      2464
lt      2441
eo      2371
ca      1909
ja      1839
la      1649
am      1292
mr      1260
fr      1222
pl      1198
sk      1156
lo      1083
se      1056
Name: count, dtype: int64